### This exercise is designed to pair with [this tutorial](https://www.kaggle.com/rtatman/bigquery-machine-learning-tutorial). If you haven't taken a look at it yet, head over and check it out first. (Otherwise these exercises will be pretty confusing!) -- Rachael 

# Stocking rental bikes

![bike rentals](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Bay_Area_Bike_Share_launch_in_San_Jose_CA.jpg/640px-Bay_Area_Bike_Share_launch_in_San_Jose_CA.jpg)

You stock bikes for a bike rental company in Austin, ensuring stations have enough bikes for all their riders. You decide to build a model to predict how many riders will start from each station during each hour, capturing patterns in seasonality, time of day, day of the week, etc.

To get started, create a project in GCP and connect to it by running the code cell below. Make sure you have connected the kernel to your GCP account in Settings.

In [1]:
# Set your own project id here
PROJECT_ID = 'gbq-kaggle-project' # a string, like 'kaggle-bigquery-240818'

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location="US")
dataset = client.create_dataset('model_dataset', exists_ok=True)

from google.cloud.bigquery import magics
from kaggle.gcp import KaggleKernelCredentials
magics.context.credentials = KaggleKernelCredentials()
magics.context.project = PROJECT_ID

In [2]:
%load_ext google.cloud.bigquery

## Linear Regression

Your dataset is quite large. BigQuery is especially efficient with large datasets, so you'll use BigQuery-ML (called BQML) to build your model. BQML uses a "linear regression" model when predicting numeric outcomes, like the number of riders.

## 1) Training vs testing

You'll want to test your model on data it hasn't seen before (for reasons described in the [Intro to Machine Learning Micro-Course](https://www.kaggle.com/learn/intro-to-machine-learning). What do you think is a good approach to splitting the data? What data should we use to train, what data should we use for test the model?

In [3]:
# You can write your notes here

#Optional splitting ratios: 60/40, 65/35, 70/30, 75/25, or 80/20

## Training data

First, you'll write a query to get the data for model-building. You can use the public Austin bike share dataset from the `bigquery-public-data.austin_bikeshare.bikeshare_trips` table. You predict the number of rides based on the station where the trip starts and the hour when the trip started. Use the `TIMESTAMP_TRUNC` function to truncate the start time to the hour.

In [4]:
# create a reference to our table
biketable = client.get_table("bigquery-public-data.austin_bikeshare.bikeshare_trips")

# look at five rows from our dataset
client.list_rows(biketable, max_results=20).to_dataframe()

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900285908,Annual Membership (Austin B-cycle),400,2014-10-26 14:12:00+00:00,2823.0,Capital Metro HQ - East 5th at Broadway,2544,East 6th & Pedernales St.,10
1,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006.0,Zilker Park West,1008,Nueces @ 3rd,39
2,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712.0,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712.0,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
4,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712.0,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
5,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541.0,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19
6,9900286143,24-Hour Kiosk (Austin B-cycle),924,2014-10-26 18:12:00+00:00,2541.0,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,17
7,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536.0,Waller & 6th St.,2536,Waller & 6th St.,6
8,9900286214,Annual Membership (Austin B-cycle),24,2014-10-26 20:12:00+00:00,2712.0,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,0
9,9900286540,24-Hour Kiosk (Austin B-cycle),117,2014-10-27 15:12:00+00:00,2536.0,Waller & 6th St.,2536,Waller & 6th St.,12


## 2) Exercise: Query the training data

Write the query to retrieve your training data. The fields should be:
1. The start_station_name
2. A time trips start, to the nearest hour. Get this with `TIMESTAMP_TRUNC(start_time, HOUR) as start_hour`
3. The number of rides starting at the station during the hour. Call this `num_rides`.
Select only the data before 2018-01-01 (so we can save data from 2018 as testing data.)

Write your query below:

In [5]:
%%bigquery 
SELECT 
  COUNT(*) AS num_rides,
  TIMESTAMP_TRUNC(start_time, HOUR) AS start_hour,
  start_station_name
FROM 
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE 
  start_time < '2018-01-01'
GROUP BY start_station_name, start_time   

,num_rides,start_hour,start_station_name
0,1,2014-10-26 14:00:00+00:00,Capital Metro HQ - East 5th at Broadway
1,1,2015-10-02 21:00:00+00:00,Zilker Park West
2,5,2014-10-26 15:00:00+00:00,Toomey Rd @ South Lamar
3,2,2014-10-26 18:00:00+00:00,State Capitol @ 14th & Colorado
4,1,2014-10-26 18:00:00+00:00,Waller & 6th St.
5,1,2014-10-26 20:00:00+00:00,Toomey Rd @ South Lamar
6,1,2014-10-27 15:00:00+00:00,Waller & 6th St.
7,1,2014-10-27 16:00:00+00:00,Main Office
8,3,2014-10-27 17:00:00+00:00,Convention Center/ 3rd & Trinity
9,1,2017-01-29 16:00:00+00:00,Pease Park


You'll want to inspect your data to ensure it looks like what you expect. Run the line below to get a quick view of the data, and feel free to explore it more if you'd like (if you don't know how to do that, the [Pandas micro-course](https://www.kaggle.com/learn/pandas)) might be helpful.

## Model creation

Now it's time to turn this data into a model. You'll use the `CREATE MODEL` statement that has a structure like: 

```sql
CREATE OR REPLACE MODEL`model_dataset.bike_trips`
OPTIONS(model_type='linear_reg') AS 
-- training data query goes here
SELECT ...
    column_with_labels AS label
    column_with_data_1 
    column_with_data_2
FROM ... 
WHERE ... (Optional)
GROUP BY ... (Optional)
```

The `model_type` and `optimize_strategy` shown here are good parameters to use in general for predicting numeric outcomes with BQML.

**Tip:** Using ```CREATE OR REPLACE MODEL``` rather than just ```CREATE MODEL``` ensures you don't get an error if you want to run this command again without first deleting the model you've created.

## 3) Exercise: Create and train the model

Below, write your query to create and train a linear regression model on the training data.

Write your query below:

In [6]:
%%bigquery
CREATE OR REPLACE MODEL `model_dataset.bike_model`
OPTIONS(model_type = 'linear_reg', optimize_strategy = 'batch_gradient_descent') as 
SELECT start_station_name,
    TIMESTAMP_TRUNC(start_time, HOUR) as start_hour,
    COUNT(*) as label
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
        WHERE start_time < '2018-01-01'
        GROUP BY start_station_name, start_time

""


In [7]:
%%bigquery
CREATE OR REPLACE MODEL`model_dataset.bike_trips`
OPTIONS(model_type='linear_reg', optimize_strategy = 'auto_strategy') AS
SELECT 
  COUNT(*) AS label,
  TIMESTAMP_TRUNC(start_time, HOUR) AS start_hour,
  start_station_name
FROM 
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE 
   start_time < '2018-01-01' 
GROUP BY 
   start_station_name, start_time    


""


In [8]:
%%bigquery
CREATE OR REPLACE MODEL`model_dataset.bike_rides`
OPTIONS(model_type='linear_reg') AS
SELECT 
  COUNT(*) AS label,
  TIMESTAMP_TRUNC(start_time, HOUR) AS start_hour,
  start_station_name
FROM 
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE 
   start_time < '2018-01-01' 
GROUP BY 
   start_station_name, start_time   

""


In [9]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `model_dataset.bike_model`)
ORDER BY iteration 

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.242578,1.206606,0.1,10998
1,0,1,1.127036,1.098010,0.2,22285
2,0,2,1.080127,1.052788,0.2,19119
3,0,3,1.058088,1.031621,0.2,20670
4,0,4,1.046263,1.020374,0.2,16194
5,0,5,1.039270,1.013156,0.2,17321


In [10]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `model_dataset.bike_trips`)
ORDER BY iteration 

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.023938,1.000527,None,12802


In [11]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `model_dataset.bike_rides`)
ORDER BY iteration 

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.023938,1.000527,None,14453


## 4) Exercise: Model evaluation

Now that you have a model, evaluate it's performance on data from 2018. 


> Note that the ML.EVALUATE function will return different metrics depending on what's appropriate for your specific model. You can just use the regular ML.EVALUATE funciton here. (ROC curves are generally used to evaluate binary problems, not linear regression, so there's no reason to plot one here.)

Write your query below:

In [12]:
%%bigquery
SELECT
  *
FROM ML.EVALUATE(MODEL `model_dataset.bike_model`, (
SELECT start_station_name,
    TIMESTAMP_TRUNC(start_time, HOUR) as start_hour,
    COUNT(*) as label
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
        WHERE start_time < '2018-01-01'
        GROUP BY start_station_name, start_time))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.431204,1.038801,0.05291,0.205013,0.213599,0.214954


In [13]:
%%bigquery
SELECT
  *
FROM ML.EVALUATE(MODEL `model_dataset.bike_trips`, (
SELECT start_station_name,
    TIMESTAMP_TRUNC(start_time, HOUR) as start_hour,
    COUNT(*) as label
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
        WHERE start_time < '2018-01-01'
        GROUP BY start_station_name, start_time))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.436758,1.023515,0.054259,0.215183,0.225171,0.225171


In [14]:
%%bigquery
SELECT
  *
FROM ML.EVALUATE(MODEL `model_dataset.bike_rides`, (
SELECT start_station_name,
    TIMESTAMP_TRUNC(start_time, HOUR) as start_hour,
    COUNT(*) as label
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
        WHERE start_time < '2018-01-01'
        GROUP BY start_station_name, start_time))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.436758,1.023515,0.054259,0.215183,0.225171,0.225171


You should see that the r^2 score here is negative. Negative values indicate that the model is worse than just predicting the mean rides for each example.

## 5) Theories for poor performance

Why would your model be doing worse than making the most simple prediction based on historical data?

In [15]:
## Thought question answer here

## 6) Exercise: Looking at predictions

A good way to figure out where your model is going wrong is to look closer at a small set of predictions. Use your model to predict the number of rides for the 22nd & Pearl station in 2018. Compare the mean values of predicted vs actual riders.

Write your query below:

In [16]:
%%bigquery

UsageError: %%bigquery is a cell magic, but the cell body is empty.


What you should see here is that the model is underestimating the number of rides by quite a bit. 

## 7) Exercise: Average daily rides per station

Either something is wrong with the model or something surprising is happening in the 2018 data. 

What could be happening in the data? Write a query to get the average number of riders per station for each year in the dataset and order by the year so you can see the trend. You can use the `EXTRACT` method to get the day and year from the start time timestamp. (You can read up on EXTRACT [in this lesson in the Intro to SQL course](https://www.kaggle.com/dansbecker/order-by)). 

Write your query below:

In [17]:
%%bigquery

UsageError: %%bigquery is a cell magic, but the cell body is empty.


## 8) What do your results tell you?

Given the daily average riders per station over the years, does it make sense that the model is failing?

In [18]:
## Thought question answer here

# 9) Next steps

Given what you've learned, what improvements do you think you could make to your model? Share your ideas on the [Kaggle Learn Forums](https://www.kaggle.com/learn-forum)! (I'll pick a couple of my favorite ideas & send the folks who shared them a Kaggle t-shirt. :)